In [ ]:
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

with open('header.json') as fi:
    credentials = json.load(fi)

header = credentials['header']

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 

parameters = {
    'start':'1',
    'limit':'5000',
    'convert':'USD'
}

session = Session()
session.headers.update(header = header)


try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd 

#This allows you to see all the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#This normalizes the data and makes it all organized in a dataframe

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

# Function to automate the data pull

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    
    parameters = {
    'start':'1',
    'limit':'15',
    'convert':'USD'
    }

    session = Session()
    session.headers.update(header = header)
    
    
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    df = df.append(df2)
    
    # Use this if you want to create a csv and append data to it
    #df = pd.json_normalize(data['data'])
    #df['timestamp'] = pd.to_datetime('now')
    #df
    
    #if not os.path.isfile(r'../notebooks/crypto_api.csv'):
        #df.to_csv(r'../notebooks/crypto_api.csv', header='column_names')
    #else:
        #df.to_csv(r'../notebooks/crypto_api.csv', mode='a', header=False)
    
    #Then to read in the file: df = pd.read_csv(r'../notebooks/crypto_api.csv')


In [ ]:
import os
from time import time
from time import sleep

for i in range(333): # We only get 333 runs per day via website protocol
    api_runner()
    print('API Run Succesfully Completed!')
    sleep(60) # Run every 60 sec / Sleep for 1 min
exit()        

In [ ]:
crypto_df = pd.read_csv("../notebooks/crypto_api.csv")
crypto_df

In [ ]:
df

# Transform Data

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

We need the columns from df3 to be rows for our visualizations, so let's do that by using stack.

In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
type(df5)

In [ ]:
df5.count()

In [ ]:
index = pd.Index(range(90))

# Set the above DataFrame index object as the index
# using set_index() function
df6 = df5.set_index(index)
df6

# If it only has the index and values try doing reset_index like "df5.reset_index()"

In [ ]:
df7 = df6.rename(columns={'level_1': 'percent_change'})

In [ ]:
df7

Changing names in the percent change column

In [ ]:
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'], ['1h','24h','7d','30d','60d','90d'])
df7

# Data Visualizations

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df10)